### Import Necessary Libraries

In [1]:
import io
import os
import glob
import pandas as pd
from datetime import datetime, timedelta

curr_date = datetime.now().strftime('%d-%m-%Y')
prev_day = (datetime.now() - timedelta(days=1)).strftime('%d-%m-%Y')

print("Current date:", curr_date)
print("Previous day:", prev_day)

Current date: 20-06-2024
Previous day: 19-06-2024


### Read files into Dataframes

In [2]:
column_names = ["S/N", "CHANNEL", "SESSION ID", "TRANSACTION TYPE", "RESPONSE", "AMOUNT",
                "TRANSACTION TIME", "ORIGINATOR INSTITUTION", "ORIGINATOR / BILLER",
                "DESTINATION INSTITUTION", "DESTINATION ACCOUNT NAME", "DESTINATION ACCOUNT NO",
                "NARRATION", "PAYMENT REFERENCE", "LAST 12 DIGITS OF SESSION_ID"]

categories = ["BILLING", "CMMS", "EBILLSPAY", "NIBSS_EASY_PAY", "NIBSS_NIP_POS", "NIBSS_NIP_EBILLSPAY",
              "NIP", "NIP DISPUTE", "NIP REVERSAL", "NQR_MERCHANT", "PRODUCTS", "NIBSS_NIP_GSI"]

# List to store the names of folders
session = []

dtype_mapping = {5: str, 11: str}

# Creating the session list
for folder_path in glob.glob("../Files/*/"):
    folder_name = os.path.basename(os.path.normpath(folder_path))
    session.append(folder_name)

dfs = []

### Data Cleaning and Transformation

In [3]:
# Iterate through each folder in the session list
for session_folder in session:
    print(f"Session Folder: {session_folder}")
    for category in categories:
        file_pattern = f"../Files/{session_folder}/{category}/*inwards successful.csv"
        file_paths = glob.glob(file_pattern)
        # print(f"File pattern for '{category}' under '{session_folder}': {file_pattern}")

        for file_path in file_paths:
            try:
                with open(file_path, 'r', encoding='utf-8') as file:
                    content = file.read()
                df = pd.read_csv(io.StringIO(content), header=None, skiprows=6, names=column_names, dtype=dtype_mapping)
                df = df.dropna(subset=df.columns[8:15], how='all')
                df = df.drop(df[df.eq(df.columns)].dropna(how='all').index)
                df["TRANSACTION TIME"] = df["TRANSACTION TIME"].str.lstrip("'")
                df["category"] = category
                df["session"] = session_folder
                dfs.append(df)
                # print(f"Appended DataFrame for file: {file_path}")
            except pd.errors.ParserError:
                print(f"Error parsing file: {file_path}. Please check the file structure.")
            except UnicodeDecodeError:
                print(f"Encoding error for file: {file_path}. Attempting to convert to UTF-8...")
                try:
                    with open(file_path, 'r', encoding='latin-1') as file:
                        content = file.read()
                    df = pd.read_csv(io.StringIO(content), header=None, skiprows=6, names=column_names, dtype=dtype_mapping)
                    df = df.dropna(subset=df.columns[8:15], how='all')
                    df = df.drop(df[df.eq(df.columns)].dropna(how='all').index)
                    df["TRANSACTION TIME"] = df["TRANSACTION TIME"].str.lstrip("'")
                    df["category"] = category
                    df["session"] = session_folder
                    dfs.append(df)
                    print(f"Successfully converted and appended DataFrame for file: {file_path}")
                except Exception as e:
                    print(f"Failed to convert file: {file_path}. Error: {str(e)}")

combined_df = pd.concat(dfs, ignore_index=True)
combined_df["S/N"] = range(1, len(combined_df) + 1)
print("Combined DataFrame is ready")

Session Folder: ECOBANK_20240619_055959
Session Folder: ECOBANK_20240619_085959
Encoding error for file: ../Files/ECOBANK_20240619_085959/NIP\NIP_050_inwards successful.csv. Attempting to convert to UTF-8...
Successfully converted and appended DataFrame for file: ../Files/ECOBANK_20240619_085959/NIP\NIP_050_inwards successful.csv
Session Folder: ECOBANK_20240619_095959
Encoding error for file: ../Files/ECOBANK_20240619_095959/NIP\NIP_050_inwards successful.csv. Attempting to convert to UTF-8...
Successfully converted and appended DataFrame for file: ../Files/ECOBANK_20240619_095959/NIP\NIP_050_inwards successful.csv
Session Folder: ECOBANK_20240619_105959
Encoding error for file: ../Files/ECOBANK_20240619_105959/NIP\NIP_050_inwards successful.csv. Attempting to convert to UTF-8...
Successfully converted and appended DataFrame for file: ../Files/ECOBANK_20240619_105959/NIP\NIP_050_inwards successful.csv
Session Folder: ECOBANK_20240619_115959
Encoding error for file: ../Files/ECOBANK_20

In [4]:
# # Iterate through each folder in the session list
# for session_folder in session:
#     print(f"Session Folder: {session_folder}")
#     for category in categories:
#         file_pattern = f"../Files/{session_folder}/{category}/*inwards successful.csv"
#         file_paths = glob.glob(file_pattern)
#         print(f"File pattern for '{category}' under '{session_folder}': {file_pattern}")

#         for file_path in file_paths:
#             try:
#                 df = pd.read_csv(file_path, header=None, skiprows=6, names=column_names, encoding='utf-8', dtype=dtype_mapping)
#                 df = df.dropna(subset=df.columns[8:15], how='all')
#                 df = df.drop(df[df.eq(df.columns)].dropna(how='all').index)
#                 df["TRANSACTION TIME"] = df["TRANSACTION TIME"].str.lstrip("'")
#                 df["category"] = category
#                 df["session"] = session_folder
#                 dfs.append(df)
#                 print(f"Appended DataFrame for file: {file_path}")
#             except pd.errors.ParserError:
#                 print(f"Error parsing file: {file_path}. Please check the file structure.")
#             except UnicodeDecodeError:
#                 print(f"Encoding error for file: {file_path}. Ignored.")

# Concatenate all DataFrames into a single DataFrame
# combined_df = pd.concat(dfs, ignore_index=True)
# combined_df["S/N"] = range(1, len(combined_df) + 1)
# print("Combined DataFrame is ready")

### Display Cleaned Data

In [5]:
display(combined_df)

,S/N,CHANNEL,SESSION ID,TRANSACTION TYPE,RESPONSE,AMOUNT,TRANSACTION TIME,ORIGINATOR INSTITUTION,ORIGINATOR / BILLER,DESTINATION INSTITUTION,DESTINATION ACCOUNT NAME,DESTINATION ACCOUNT NO,NARRATION,PAYMENT REFERENCE,LAST 12 DIGITS OF SESSION_ID,category,session
0,1,BANK TELLER,'999999240619021321367936388904,Fund Transfer (Direct Credit),Approved or Completed Successfully,20000.00,2024-06-19 02:13:24,STERLING BANK,LAVENDER MICROFINANCE BANK,ECOBANK,CAC COVENANT OF MERCY,2080047985,125931383876,'NIPMINI/090271240619021320401101354474/090271...,'367936388904,NIBSS_EASY_PAY,ECOBANK_20240619_055959
1,2,BANK TELLER,'999999240619000028337658960126,Fund Transfer (Direct Credit),Approved or Completed Successfully,15000.00,2024-06-19 00:00:29,OPTIMUS BANK,KC Gaming Networks Limited,ECOBANK,KPODEZIAHOU COMLAN ANGELOT,5051108434,BET9JA WITHDRAWAL,'NIPMINI/090601240619000028572456672734/382c57...,'337658960126,NIBSS_EASY_PAY,ECOBANK_20240619_055959
2,3,BANK TELLER,'999999240619000154355588218988,Fund Transfer (Direct Credit),Approved or Completed Successfully,2000.00,2024-06-19 00:01:54,OPTIMUS BANK,KC Gaming Networks Limited,ECOBANK,JOSEPH EBIEGBERI SYLVESTER,0201081300,BET9JA WITHDRAWAL,'NIPMINI/090601240619000153672126457553/9619e5...,'355588218988,NIBSS_EASY_PAY,ECOBANK_20240619_055959
3,4,BANK TELLER,'999999240619000228323941512752,Fund Transfer (Direct Credit),Approved or Completed Successfully,7000.00,2024-06-19 00:02:29,OPTIMUS BANK,KC Gaming Networks Limited,ECOBANK,ANGALABIRI RICHARD,3591020160,BET9JA WITHDRAWAL,'NIPMINI/090601240619000228574733436214/d19c91...,'323941512752,NIBSS_EASY_PAY,ECOBANK_20240619_055959
4,5,BANK TELLER,'999999240619000415251476697809,Fund Transfer (Direct Credit),Approved or Completed Successfully,100000.00,2024-06-19 00:04:16,OPTIMUS BANK,KC Gaming Networks Limited,ECOBANK,OGODU ROMANUS,2543009659,BET9JA WITHDRAWAL,'NIPMINI/090601240619000415775788624534/4f75d0...,'251476697809,NIBSS_EASY_PAY,ECOBANK_20240619_055959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455454,455455,INTERNET BANKING,'000006240613182911447561867257,Fund Transfer (Direct Credit),Approved or Completed Successfully,100000.00,2024-06-13 18:29:08,JAIZ BANK,NURUDEEN OLASUNBO ISHOLA,ECOBANK,folashade Raheemat,8106467195,IFO folashade Raheemat,'JF0999145915,'447561867257,NIP REVERSAL,ECOBANK_20240619_235959
455455,455456,MOBILE PHONE,'000013240613112946000136219940,Fund Transfer (Direct Credit),Approved or Completed Successfully,1000.00,2024-06-13 11:29:52,GTBANK,WONAH MOSES OGAR,ECOBANK,WONAH MOSES OGAR,3132104634,NaN,'000136219940,'000136219940,NIP REVERSAL,ECOBANK_20240619_235959
455456,455457,MOBILE PHONE,'000013240613205214000137772224,Fund Transfer (Direct Credit),Approved or Completed Successfully,200000.00,2024-06-13 20:52:15,GTBANK,OMELEBELE CHIBUZOR STEPHANIE,ECOBANK,OLUWAKEMI FATIMAH ODUGBESI,5390097949,NaN,'000137772224,'000137772224,NIP REVERSAL,ECOBANK_20240619_235959
455457,455458,MOBILE PHONE,'000013240613212214000137838034,Fund Transfer (Direct Credit),Approved or Completed Successfully,10000.00,2024-06-13 21:22:15,GTBANK,ALIH BLESSING,ECOBANK,ALIH BLESSING,5321070690,NaN,'000137838034,'000137838034,NIP REVERSAL,ECOBANK_20240619_235959


### Write Data back into the Output Directory

In [6]:
output_dir = "../Output"
os.makedirs(output_dir, exist_ok=True)

In [7]:
# Generate the output file path with the current date
output_file_path = os.path.join(output_dir, f"Inwards successful_{prev_day}.csv")
# output_file_path = os.path.join(output_dir, f"Inwards successful_17-05-2024.csv")
combined_df.to_csv(output_file_path, index=False, mode='w')

print(f"Output file saved to: {output_file_path}")

Output file saved to: ../Output\Inwards successful_19-06-2024.csv
